In [ ]:
# !pip install transformers==3.5.0

In [1]:
import os
import time
import datetime
import torch
from torch.utils.data import Dataset


import os
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

import argparse
import logging
import os
import random

import numpy as np

import torch
from torch.utils.data import DataLoader

from transformers import (
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)

In [2]:
from transformers import PegasusTokenizer,PegasusForConditionalGeneration
from rouge_score import rouge_scorer
import torch
import os
from tqdm.auto import tqdm
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hemant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
check_path = "AMICorpusXML/data/ami-summary/abstractive/ES2004a.abssumm.txt"
from pathlib import Path

my_file = Path(check_path)
if not my_file.is_file():
    if not Path("AMICorpusXML/data").is_dir():
        ! git clone https://github.com/gcunhase/AMICorpusXML
    ! python /content/AMICorpusXML/main_obtain_meeting2summary_data.py --summary_type abstractive
else:
    print("Data already prepared... Importing\n")

Data already prepared... Importing



In [4]:
# taking same split to make easy comparision
# data split knowledge: http://groups.inf.ed.ac.uk/ami/corpus/datasets.shtml
# 5 x 4 = 20 
test = "ES2004, ES2014, IS1009, TS3003, TS3007".split(',')

val = "ES2003, ES2011, IS1008, TS3004, TS3006".split(',')
train = """ES2002, ES2005, ES2006, ES2007, ES2008, ES2009, ES2010, ES2012, ES2013, ES2015, 
            ES2016, IS1000, IS1001, IS1003, IS1004, IS1006, IS1007, TS3005, TS3008, TS3009, TS3010, 
            TS3011, TS3012""".split(",")


def data_create(path):
    story = []
    story_directory = r'AMICorpusXML/data/ami-transcripts-stories/abstractive'
    for filename in os.listdir(story_directory):
        if filename.endswith(".story"):
            for each in path:
                if each.strip() in str(filename):
                    story.append(filename)
    summary = []
    sum_directory = r'AMICorpusXML/data/ami-summary/abstractive'
    for filename in os.listdir(sum_directory):
        if filename.endswith(".txt"):
            for each in path:
                if each.strip() in str(filename):
                    summary.append(filename)
    return [data_from_file(f"AMICorpusXML/data/ami-transcripts-stories/abstractive/{story[i]}", f"AMICorpusXML/data/ami-summary/abstractive/{summary[i]}") for i in range(len(story))]

In [8]:
def prepare_model(model_name = 'google/pegasus-xsum'):
    print("----------------- Preparing Model -----------------")
    
    torch_device = "cpu" #'cuda'if torch.cuda.is_available() else 'cpu'
    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
    print("----------------- Model Prepared -----------------")
    return tokenizer, model, torch_device

def test_model(src_text, tokenizer, model):
#     print("\nNew Testing started...")
    batch = tokenizer.prepare_seq2seq_batch(src_text,max_length=512).to(torch_device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

def data_from_file(f1,f2):
    with open(f1, 'r') as file:
        data1 = file.read().replace('\n', '')
    with open(f2, 'r') as file:
        data2 = file.read().replace('\n', '')
    return data1, data2

In [9]:
# taking same split to make easy comparision
# data split knowledge: http://groups.inf.ed.ac.uk/ami/corpus/datasets.shtml
# 5 x 4 = 20 
test = "ES2004, ES2014, IS1009, TS3003, TS3007".split(',')

val = "ES2003, ES2011, IS1008, TS3004, TS3006".split(',')
train = """ES2002, ES2005, ES2006, ES2007, ES2008, ES2009, ES2010, ES2012, ES2013, ES2015, 
            ES2016, IS1000, IS1001, IS1003, IS1004, IS1006, IS1007, TS3005, TS3008, TS3009, TS3010, 
            TS3011, TS3012""".split(",")

story = []
story_directory = r'AMICorpusXML/data/ami-transcripts-stories/abstractive'
for filename in os.listdir(story_directory):
    if filename.endswith(".story"):
        for each in test:
            if each.strip() in str(filename):
                story.append(filename)
     
summary = []
sum_directory = r'AMICorpusXML/data/ami-summary/abstractive'
for filename in os.listdir(sum_directory):
    if filename.endswith(".txt"):
        for each in test:
            if each.strip() in str(filename):
                summary.append(filename)

df = pd.DataFrame()
st, sums_pred, sums_ann = [], [], []

# remove later
# flag = 0

# tokenizer, model, torch_device = prepare_model()
model.eval()
for each in tqdm(story):
    a = each.split('.')[0]
    story_file = story_directory+'/'+each
    summ_file = sum_directory+'/'+a+".abssumm.txt"
    story_file, summ_file = data_from_file(story_file, summ_file)
    st.append(story_file)
    src_text = [story_file.replace('\n',' ')]
    result = test_model(src_text, tokenizer, model)
    sums_pred.append(result[0])
    sums_ann.append(summ_file)

    # flag+=1
    # if flag >=2: break

df['stories'] = st
df['summaries'] = sums_pred
df['Summary annotated'] = sums_ann
df.to_csv('summary_df.csv')
display(df.head())


test# R1 and R2 for a sample summary 
a = df["stories"][0]
b = df["summaries"][0]
c = df["Summary annotated"][0]

def rogue_sc(b, a):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
    scores = scorer.score(b,a)
    return scores

scores = rogue_sc(b,a)
print("Word Capture: ", scores['rouge1'][1])
scores = rogue_sc(b,c)
print("ROUGE-1 Score: ", scores['rouge1'][1])
print("ROUGE-2 Score: ", scores['rouge2'][1])

import numpy as np

rouge1 = []
rouge2 = []
word_capt = []
for index, row in df.iterrows():
    rouge1.append(rogue_sc(row['summaries'], row['Summary annotated'])['rouge1'][1])
    rouge2.append(rogue_sc(row['summaries'], row['Summary annotated'])['rouge2'][1])
    word_capt.append(rogue_sc(row['summaries'], row['Summary annotated'])['rouge1'][1])
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))

----------------- Preparing Model -----------------
----------------- Model Prepared -----------------



New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...



,stories,summaries,Summary annotated
0,Okay. About the components design. Um for the ...,Here's what we know so far about Apple's docki...,The project manager opened the meeting and rec...
1,Okay. Everybody found his place again? Yeah? T...,Here's the full meeting transcript:,The project manager stated the agenda and the ...
2,"Okay, is everybody ready? Mm-hmm. Um I take it...",Mm-hmm.,The meeting begins and the marketing expert st...
3,Well hi everyone again. Um like before we uh I...,This is the meeting where we discuss the desig...,The project manager opens the meeting by going...
4,So is Why not save that. Do you want to replac...,"So I've tried to transfer it to My Documents, ...",The first prototype for the remote control was...


Word Capture:  0.9166666666666666
ROUGE-1 Score:  0.0
ROUGE-2 Score:  0.0
Average Rouge1 Score:  35.14517243762526
Average Rouge2 Score:  9.775370970492922


In [10]:
def label_smoothed_nll_loss(lprobs, target, epsilon, ignore_index=-100):
    """From fairseq"""
    if target.dim() == lprobs.dim() - 1:
        target = target.unsqueeze(-1)
    nll_loss = -lprobs.gather(dim=-1, index=target)
    smooth_loss = -lprobs.sum(dim=-1, keepdim=True)
    if ignore_index is not None:
        pad_mask = target.eq(ignore_index)
        nll_loss.masked_fill_(pad_mask, 0.0)
        smooth_loss.masked_fill_(pad_mask, 0.0)
    else:
        nll_loss = nll_loss.squeeze(-1)
        smooth_loss = smooth_loss.squeeze(-1)

    nll_loss = nll_loss.sum()  # mean()? Scared to break other math.
    smooth_loss = smooth_loss.sum()
    eps_i = epsilon / lprobs.size(-1)
    loss = (1.0 - epsilon) * nll_loss + eps_i * smooth_loss
    return loss, nll_loss


In [11]:
num_train_epochs = 5
#model.resize_token_embeddings(len(tokenizer))
weight_decay =0.0
learning_rate = 1e-4 
adam_epsilon = 1e-8
warmup_steps = 0
t_total = (607276 // 2 ) 

# dhak chiki dhak

In [12]:
test_ = data_create(test)
train_ = data_create(train)
val_ = data_create(val)

In [13]:
import json
with open ('../Ext_Abs/train.txt', "r") as f:
    train_ = json.loads(f.read())

In [14]:
# a = []
# for i in test_:
#     a.append(len(i[0].split(' ')))

In [15]:
from transformers.modeling_bart import shift_tokens_right

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [16]:
model_name = 'google/pegasus-xsum'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
output_dir = 'data/train/'
# tokenizer = PegasusTokenizer.from_pretrained(output_dir)
# model = PegasusForConditionalGeneration.from_pretrained(output_dir).to(torch_device)

In [17]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
    )
# optimizer.load_state_dict(torch.load(os.path.join(output_dir, 'optimizer.pt')))
# scheduler.load_state_dict(torch.load(os.path.join(output_dir, 'scheduler.pt')))

In [ ]:
train_ = [train_[:] for i in range(len(train_))]

In [31]:
for i, b in enumerate(model.parameters()):
    if i == 400: break
    b.requires_grad = False

pad_token_id = tokenizer.pad_token_id
epochs = 5
m = []
batch_size = 5
train_loader = [train_[i:i+batch_size] for i in range(0,len(train_),batch_size)]
for epoc in range(epochs):
  t0 = time.time()
  print("")
  print('======== Epoch {} ========'.format(epoc+1))
  model.train()
  total_train_loss = 0
  for i,batch in tqdm(enumerate(train_loader)):
    title = [j[1] for j in batch]
    body = [j[0] for j in batch]
        
#     title = [batch[1]]
#     body = [batch[0]]

    batch_tokens = tokenizer.prepare_seq2seq_batch(body,title,max_length=512,max_target_length=202,padding='longest').to(device)
#     print(batch_tokens['input_ids'].shape, len(body[0].split()))
    m.append(batch_tokens)
    decoder_input_ids = shift_tokens_right(batch_tokens['labels'], pad_token_id)
    outputs = model(batch_tokens['input_ids'], attention_mask=batch_tokens['attention_mask'], decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]
    lprobs = torch.nn.functional.log_softmax(lm_logits, dim=-1)
    loss, nll_loss = label_smoothed_nll_loss(
                lprobs, batch_tokens['labels'],0.1, ignore_index=pad_token_id
            )
    total_train_loss += loss.item()
  
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    if i%5 == 0:
      print("batch :" + str(i)+ "Training Loss: " +str(total_train_loss))
    if (i+1) % 10000 == 0:
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        torch.save(optimizer.state_dict(), os.path.join(output_dir, 'optimizer.pt'))
        torch.save(scheduler.state_dict(), os.path.join(output_dir, 'scheduler.pt')) 
  training_time = format_time(time.time() - t0)
  avg_train_loss = total_train_loss / 1050994
  print("traing loss epoch"+str(epoc+1)+":"+str(avg_train_loss))
  print("time : {}".format(training_time))
  
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  model_to_save.save_pretrained(output_dir)
  tokenizer.save_pretrained(output_dir)
  torch.save(optimizer.state_dict(), os.path.join(output_dir, 'optimizer.pt'))
  torch.save(scheduler.state_dict(), os.path.join(output_dir, 'scheduler.pt'))


======== Epoch 1 ========


batch :0Training Loss: 3382.04150390625
batch :5Training Loss: 21117.46435546875
batch :10Training Loss: 37073.857177734375
batch :15Training Loss: 53470.7646484375

traing loss epoch1:0.05916025193694612
time : 0:00:08

======== Epoch 2 ========


/media/data_dump/hemant/hemant/nlp/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


batch :0Training Loss: 3295.244384765625
batch :5Training Loss: 21043.076904296875
batch :10Training Loss: 36996.321044921875
batch :15Training Loss: 53402.813232421875

traing loss epoch2:0.05915286147231514
time : 0:00:08

======== Epoch 3 ========


batch :0Training Loss: 3269.593017578125
batch :5Training Loss: 20736.5830078125
batch :10Training Loss: 36491.4619140625
batch :15Training Loss: 52623.07177734375

traing loss epoch3:0.05816053125399919
time : 0:00:08

======== Epoch 4 ========


batch :0Training Loss: 3239.453857421875
batch :5Training Loss: 20798.3994140625
batch :10Training Loss: 36493.6494140625
batch :15Training Loss: 52407.1533203125

traing loss epoch4:0.057997312790357566
time : 0:00:08

======== Epoch 5 ========


batch :0Training Loss: 3214.54931640625
batch :5Training Loss: 20718.232177734375
batch :10Training Loss: 36391.16845703125
batch :15Training Loss: 52411.671630859375

traing loss epoch5:0.058038490675255995
time : 0:00:08


In [32]:
# taking same split to make easy comparision
# data split knowledge: http://groups.inf.ed.ac.uk/ami/corpus/datasets.shtml
# 5 x 4 = 20 
test = "ES2004, ES2014, IS1009, TS3003, TS3007".split(',')

val = "ES2003, ES2011, IS1008, TS3004, TS3006".split(',')
train = """ES2002, ES2005, ES2006, ES2007, ES2008, ES2009, ES2010, ES2012, ES2013, ES2015, 
            ES2016, IS1000, IS1001, IS1003, IS1004, IS1006, IS1007, TS3005, TS3008, TS3009, TS3010, 
            TS3011, TS3012""".split(",")

story = []
story_directory = r'AMICorpusXML/data/ami-transcripts-stories/abstractive'
for filename in os.listdir(story_directory):
    if filename.endswith(".story"):
        for each in test:
            if each.strip() in str(filename):
                story.append(filename)
     
summary = []
sum_directory = r'AMICorpusXML/data/ami-summary/abstractive'
for filename in os.listdir(sum_directory):
    if filename.endswith(".txt"):
        for each in test:
            if each.strip() in str(filename):
                summary.append(filename)

df = pd.DataFrame()
st, sums_pred, sums_ann = [], [], []

# remove later
# flag = 0
# torch_device = "cpu"
model.eval()

def test_model(src_text, tokenizer, model):
    print("\nNew Testing started...")
    batch = tokenizer.prepare_seq2seq_batch(src_text,max_length=512).to(torch_device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

for each in tqdm(story):
    a = each.split('.')[0]
    story_file = story_directory+'/'+each
    summ_file = sum_directory+'/'+a+".abssumm.txt"
    story_file, summ_file = data_from_file(story_file, summ_file)
    st.append(story_file)
    src_text = [story_file.replace('\n',' ')]
    result = test_model(src_text, tokenizer, model)
    sums_pred.append(result[0])
    sums_ann.append(summ_file)

    # flag+=1
    # if flag >=2: break


New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...

New Testing started...



In [33]:

df['stories'] = st
df['summaries'] = sums_pred
df['Summary annotated'] = sums_ann
df.to_csv('summary_df.csv')
display(df.head())


test# R1 and R2 for a sample summary 
a = df["stories"][0]
b = df["summaries"][0]
c = df["Summary annotated"][0]

def rogue_sc(b, a):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
    scores = scorer.score(b,a)
    return scores

scores = rogue_sc(b,a)
print("Word Capture: ", scores['rouge1'][1])
scores = rogue_sc(b,c)
print("ROUGE-1 Score: ", scores['rouge1'][1])
print("ROUGE-2 Score: ", scores['rouge2'][1])

import numpy as np

rouge1 = []
rouge2 = []
word_capt = []
for index, row in df.iterrows():
    rouge1.append(rogue_sc(row['summaries'], row['Summary annotated'])['rouge1'][1])
    rouge2.append(rogue_sc(row['summaries'], row['Summary annotated'])['rouge2'][1])
    word_capt.append(rogue_sc(row['summaries'], row['Summary annotated'])['rouge1'][1])
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))

,stories,summaries,Summary annotated
0,Okay. About the components design. Um for the ...,The project manager opens the meeting by expla...,The project manager opened the meeting and rec...
1,Okay. Everybody found his place again? Yeah? T...,The project manager opens the meeting by stati...,The project manager stated the agenda and the ...
2,"Okay, is everybody ready? Mm-hmm. Um I take it...",The project manager opens the meeting by stati...,The meeting begins and the marketing expert st...
3,Well hi everyone again. Um like before we uh I...,The marketing expert and industrial designer p...,The project manager opens the meeting by going...
4,So is Why not save that. Do you want to replac...,The project manager opens the meeting by stati...,The first prototype for the remote control was...


Word Capture:  0.9473684210526315
ROUGE-1 Score:  0.6491228070175439
ROUGE-2 Score:  0.32142857142857145
Average Rouge1 Score:  54.39812172079005
Average Rouge2 Score:  18.367252485579392


In [ ]:
# Word Capture:  0.9310344827586207
# ROUGE-1 Score:  0.5344827586206896
# ROUGE-2 Score:  0.24561403508771928
# Average Rouge1 Score:  53.47327685825854
# Average Rouge2 Score:  16.957205303947593